# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Подключимся к базе данных**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Ознакомимся с базами**

In [2]:
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [3]:
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Задание 1: Посчитайте, сколько книг вышло после 1 января 2000 года.

In [6]:
query = '''SELECT COUNT(*) 
           FROM books 
           WHERE publication_date > '2000-01-01'
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


819 книг вышло после 1 января 2000 года.

### Задание 2: Для каждой книги посчитайте количество обзоров и среднюю оценку

In [7]:
query = '''SELECT books.title,
           COUNT(DISTINCT reviews.review_id) AS review_count,
           AVG(ratings.rating) AS average_rating
           FROM books
           LEFT JOIN reviews ON books.book_id = reviews.book_id
           LEFT JOIN ratings ON books.book_id = ratings.book_id
           GROUP BY books.book_id;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,review_count,average_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


Посчитали для каждой книги количество обзоров и среднюю оценку.

### Задание 3: Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [8]:
query = '''SELECT publishers.publisher, COUNT(*) AS book_count
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY book_count DESC
LIMIT 1;
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,book_count
0,Penguin Books,42


По результату запроса, получили издательство Penguin Books, у которого 42 книги.

### Задание 4: Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [9]:
query = '''
WITH 
high_rating AS (SELECT book_id,
                       COUNT(rating),
                       AVG(rating) AS avg_rating
                FROM ratings
                GROUP BY book_id
                HAVING COUNT(rating) >= 50)
SELECT a.author_id,
       a.author,
       ROUND(AVG(avg_rating),2) AS avg_rating
FROM high_rating AS hr
LEFT JOIN books AS b ON hr.book_id = b.book_id
LEFT JOIN authors AS a ON b.author_id = a.author_id
GROUP BY a.author_id
ORDER BY AVG(avg_rating) DESC
LIMIT 1
'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.28


По результату запроса получили автора с самой высокой средней оценкой книг. Это .K. Rowling/Mary GrandPré, с рейтингом 4.28.

### Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [10]:
query = '''SELECT AVG(review_count) AS avg_review_count
FROM (
SELECT ratings.username, COUNT(DISTINCT reviews.review_id) AS review_count
FROM ratings
JOIN reviews ON ratings.book_id = reviews.book_id AND ratings.username = reviews.username
WHERE ratings.username IN (
SELECT username FROM (
SELECT username, COUNT(rating_id) AS rating_count
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48
) top_raters
)
GROUP BY ratings.username
) user_reviews
           
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review_count
0,24.0


Мы посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок. Значение - 24.